In [ ]:
#Install the needed libraries
!pip3 install bs4
!pip3 install pandas
!pip3 install selenium
!pip3 install boto3
!pip3 install psycopg2-binary

In [ ]:
#Import the needed libraries for this project
import requests
import pandas as pd
import time
import re
import psycopg2
import boto3
from datetime import date
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from io import StringIO 

In [ ]:
#To get the data from Spotify we will be sending the requests to the Spotify API
#We will not be using the spotipy library, instead all requests will be made manually 

In [ ]:
#Get the access token needed to make the API call and use the access token to make a request to the Spotify API


#Get the client id and client secret from Spotify
CLIENT_ID = '85b72157af5448498f1374633c01162e'
CLIENT_SECRET = '7e871a9f34524944a3f1908549c2d3a6'

#Send a post request to the auth endpoint to get the client credentials 
AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

#Convert the response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Define the headers for making the requests
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

#Get the base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [ ]:
#Use the access token to make a request to the Spotify API

#Define the headers for making the requests
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

#Get the base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [ ]:
#Make a request to the get playlist items endpoint to get the playlists information

#Get the playlist id
playlist_id='37i9dQZEVXbMDoHDwVN2tF'

#Send a get requests to the formatted endpoint to get the playlist items
r = requests.get(BASE_URL + 'playlists/' + playlist_id +'/tracks', headers=headers)
r = r.json()

In [ ]:
#Clean up the request and get the following information artist name, artist id, song name, song id
#Store data in a dataframe

#Declare the dataframe with the defined columns
top_50_df = pd.DataFrame(columns = ["song_name", "song_id", "artist_name","artist_id"])

playlist_items=r['items']

for item in playlist_items:
    song_name=item['track']['name']
    song_id=item['track']['id']
    for artist in item['track']['album']['artists']:
        artist_id=artist['id']
        artist_name=artist['name']
        new_row = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist_name':artist_name, 'artist_id':artist_id},index=[0])
        top_50_df = pd.concat([new_row,top_50_df.loc[:]]).reset_index(drop=True)
top_50_df

In [ ]:
#Get the number of times this song has been played 
#This information is not available in the Spotify API and will be scraped from the Spotify web page
options = Options()
options.headless = True
options.add_argument("--window-size=1920,3000")

driver = webdriver.Chrome(options=options, executable_path='/Users/sidm7184/Downloads/chromedriver_mac_arm64/chromedriver')
driver.get("https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF")
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
songs_with_plays=soup.find_all('div',{'role':'row'})[1::]

records={}
for song in songs_with_plays:
    songname=song.find('div',{'class':'Type__TypeElement-sc-goli3j-0 kHHFyx t_yrXoUO3qGsJS4Y6iXX standalone-ellipsis-one-line'}).get_text()
    plays=song.find('div',{'Type__TypeElement-sc-goli3j-0 hGXzYa nYg_xsOVmrVE_8qk1GCW'}).get_text()
    records[songname]=plays.replace(",", "")
top_50_df['plays'] = top_50_df['song_name'].map(records) 
top_50_df

In [ ]:
#Get the information on the songs audio features
song_ids=top_50_df['song_id'].to_list()
song_ids=list(set(song_ids))

song_features=[]

for song in song_ids:
    audio_feats = requests.get(BASE_URL + 'audio-features/'+ song, headers=headers)
    audio_feats = audio_feats.json()
    
    song_features.append(audio_feats)
song_features_df=pd.DataFrame(song_features)
song_features_df
top_50_df = pd.merge(left=top_50_df, right=song_features_df, left_on='song_id', right_on='id')
top_50_df=top_50_df.drop(['id', 'uri','track_href','analysis_url','type','time_signature'], axis=1)
top_50_df

In [ ]:
date=date.today()
top_50_df['date'] =[date for i in range(len(top_50_df))]
top_50_df['month'] =[date.month for i in range(len(top_50_df))]
top_50_df['month_name'] =[date.strftime("%B") for i in range(len(top_50_df))]
top_50_df['day_of_month'] =[date.day for i in range(len(top_50_df))]
top_50_df['day_of_week'] =[date.today().weekday() for i in range(len(top_50_df))]
top_50_df['day_name'] =[date.strftime("%A") for i in range(len(top_50_df))]
top_50_df['quarter'] =[((date.month)//3)+1 for i in range(len(top_50_df))]

top_50_df

In [ ]:
#put the dataframe into an s3 bucket as a csv
s3 = boto3.resource("s3")
csv_buffer = StringIO()
top_50_df.to_csv(csv_buffer)
bucket_name = 'spotify-etl'
s3_object_name = '{} Top 50 Songs Global'.format(date.today())
s3.Object(bucket_name, s3_object_name).put(Body=csv_buffer.getvalue())
print("Dataframe is saved as CSV in S3 bucket.")

In [ ]:
#Connect to postgreSQL
print(list(top_50_df))
top_50_df

In [ ]:
try:
    #connect to the default database
    conn = psycopg2.connect(database="postgres", user='postgres', password='postgres', host='localhost', port= '5432')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create a sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS playlistdata;")
    cur.execute("CREATE DATABASE playlistdata;")
    
    #close default database
    conn.close()
    
    #connect to sparkify database
    conn = psycopg2.connect(database="playlistdata", user='postgres', password='postgres', host='localhost', port= '5432')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    print("Connection Successful")
except psycopg2.Error as e:
    print("Error could not connect to the database")
    print(e)

In [ ]:
#Query for making the songs table
create_songs_table="""
CREATE TABLE songs (
  song_id TEXT,
  name TEXT,
  danceability numeric (10,2),
  energy numeric (10,2),
  key numeric (10,2),
  loudness numeric (10,2),
  mode numeric (10,2),
  speechiness numeric (10,2),
  acousticness numeric (10,2),
  instrumentalness numeric (10,2),
  liveness numeric (10,2),
  valence numeric (10,2),
  tempo numeric (10,2),
  duration_ms INTEGER,
  PRIMARY KEY (song_id)
);
"""
#Query for making the artists table
create_artists_table='''
CREATE TABLE artists (
  artist_id TEXT,
  name TEXT NOT NULL,
  PRIMARY KEY (artist_id)
);
'''
#Query for making the plays table
create_song_artist_table="""
CREATE TABLE song_artists (
  song_id TEXT REFERENCES songs (song_id),
  artist_id TEXT REFERENCES artists (artist_id),
  PRIMARY KEY (song_id,artist_id)
);
"""
#Query for making the calendar table
create_calendar_table='''
CREATE TABLE calendar (
  date DATE,
  month INTEGER,
  month_name TEXT,
  day_of_month INTEGER,
  day_of_week INTEGER,
  day_name TEXT,
  quarter INTEGER,
  PRIMARY KEY (date)
);
'''
#Query for making the plays table
create_plays_table="""
CREATE TABLE plays (
  song_id TEXT REFERENCES songs (song_id),
  date DATE REFERENCES calendar (date),
  song_plays numeric,
  PRIMARY KEY (song_id,date)
);
"""
cur.execute(create_songs_table)
cur.execute(create_artists_table)
cur.execute(create_song_artist_table)
cur.execute(create_calendar_table)
cur.execute(create_plays_table)
print('Success')

In [ ]:
songs_df=top_50_df[['song_id','song_name','danceability','energy','key', 
                 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
                 'liveness', 'valence', 'tempo', 'duration_ms']]

songs_df=songs_df.drop_duplicates(subset='song_id', keep="last").reset_index(drop=True)
songs_df

In [ ]:
artists_df=top_50_df[['artist_id','artist_name']]
artists_df=artists_df.drop_duplicates().reset_index(drop=True)
artists_df

In [ ]:
song_artist_df=top_50_df[['song_id','artist_id']]
song_artist_df

In [ ]:
calendar_df=top_50_df[['date', 'month', 'month_name', 'day_of_month', 'day_of_week', 'day_name', 'quarter']]
calendar_df=calendar_df.drop_duplicates().reset_index(drop=True)
calendar_df

In [ ]:
plays_df=top_50_df[['song_id','date','plays']]
plays_df=plays_df.drop_duplicates().reset_index(drop=True)
plays_df

In [ ]:
songs_insert='''
INSERT INTO songs(
  song_id,
  name,
  danceability,
  energy,
  key,
  loudness,
  mode,
  speechiness,
  acousticness,
  instrumentalness,
  liveness,
  valence,
  tempo,
  duration_ms
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
'''
#Insert the data
for i, row in songs_df.iterrows():
    cur.execute(songs_insert,list(row))
conn.commit()
print("Success")

In [ ]:
artist_insert='''
INSERT INTO artists(
  artist_id,
  name
    ) VALUES (%s,%s);
'''
#Insert the data
for i, row in artists_df.iterrows():
    cur.execute(artist_insert,list(row))
conn.commit()
print("Success")

In [ ]:
song_artist_insert='''
INSERT INTO song_artists(
  song_id,
  artist_id 
    ) VALUES (%s,%s);
'''
#Insert the data
for i, row in song_artist_df.iterrows():
    cur.execute(song_artist_insert,list(row))
conn.commit()
print("Success")

In [ ]:
calendar_insert='''
INSERT INTO calendar(
  date,
  month,
  month_name,
  day_of_month,
  day_of_week,
  day_name,
  quarter
    ) VALUES (%s,%s,%s,%s,%s,%s,%s);
'''
#Insert the data
for i, row in calendar_df.iterrows():
    cur.execute(calendar_insert,list(row))
conn.commit()
print("Success")

In [ ]:
plays_insert='''
INSERT INTO plays(
  song_id,
  date,
  song_plays
    ) VALUES (%s,%s,%s);
'''
#Insert the data
for i, row in plays_df.iterrows():
    cur.execute(plays_insert,list(row))
conn.commit()
print("Success")